In [1]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.units import mm
from reportlab.lib import colors
import random
import os

# ================= CONFIGURAÇÕES GERAIS =================
# Definição EXATA das 13 colunas
COL_WIDTHS = [
    22*mm, # 0: Grupo
    18*mm, # 1: Pk F
    14*mm, # 2: Sq F
    28*mm, # 3: CL F (Nome)
    14*mm, # 4: CL F (Dial)
    28*mm, # 5: CR F (Nome)
    14*mm, # 6: CR F (Dial)
    18*mm, # 7: Pk T
    14*mm, # 8: Sq T
    28*mm, # 9: CL T (Nome)
    14*mm, # 10: CL T (Tempo)
    24*mm, # 11: CR T (Tempo - Coluna Única)
    18*mm  # 12: TM
]
ROW_HEIGHT = 16*mm 

def desenhar_cabecalho(c, width, height):
    """Desenha o cabeçalho fake da empresa"""
    c.setFont("Helvetica-Bold", 14)
    c.drawString(15*mm, height - 20*mm, "ENERGYCORP S.A. - DISTRIBUIÇÃO")
    
    c.setFont("Helvetica-Bold", 10)
    c.setFillColor(colors.lightgrey)
    c.rect(15*mm, height - 35*mm, width - 30*mm, 8*mm, fill=1, stroke=0)
    c.setFillColor(colors.black)
    c.drawCentredString(width/2, height - 31*mm, "SUMÁRIO DE AJUSTES DE RELIGADOR (MODELO FICTÍCIO)")
    
    c.setFont("Helvetica", 9)
    y_data = height - 45*mm
    c.drawString(15*mm, y_data, f"Alimentador: ALIM-{random.randint(100,999)}")
    c.drawString(120*mm, y_data, f"Coord.: {random.randint(10,99)} {random.randint(100000,999999)}:{random.randint(1000000,9999999)}")
    c.drawString(width - 60*mm, y_data, f"Data: {random.randint(1,28)}/0{random.randint(1,9)}/2024")
    
    c.setStrokeColor(colors.black)
    c.setLineWidth(1)
    c.line(15*mm, y_data - 5*mm, width - 15*mm, y_data - 5*mm)

def desenhar_grade_grupos123(c, x_start, y_start):
    """Desenha a estrutura da grade"""
    total_w = sum(COL_WIDTHS)
    y_bottom = y_start - (5 * ROW_HEIGHT)
    
    c.setLineWidth(0.5)
    c.setStrokeColor(colors.black)
    c.rect(x_start, y_bottom, total_w, 5 * ROW_HEIGHT)
    
    # --- Linhas Horizontais Principais ---
    for i in range(1, 5):
        y = y_start - (i * ROW_HEIGHT)
        c.line(x_start, y, x_start + total_w, y)

    # --- Linhas Horizontais Intermediárias (Sub-divisões) ---
    y_data_start = y_start - (2 * ROW_HEIGHT)
    for i in range(3):
        y_mid = y_data_start - (i * ROW_HEIGHT) - (ROW_HEIGHT / 2)
        
        # FASE (Cols 3 a 6)
        x_fase = x_start + sum(COL_WIDTHS[:3])
        c.line(x_fase, y_mid, x_fase + sum(COL_WIDTHS[3:7]), y_mid)
        # TERRA CL (Cols 9 e 10)
        x_terra_cl = x_start + sum(COL_WIDTHS[:9])
        c.line(x_terra_cl, y_mid, x_terra_cl + COL_WIDTHS[9] + COL_WIDTHS[10], y_mid)

    # --- Linhas Verticais ---
    c.line(x_start + COL_WIDTHS[0], y_start, x_start + COL_WIDTHS[0], y_bottom)
    c.line(x_start + sum(COL_WIDTHS[:7]), y_start, x_start + sum(COL_WIDTHS[:7]), y_bottom)
    c.line(x_start + sum(COL_WIDTHS[:12]), y_start, x_start + sum(COL_WIDTHS[:12]), y_bottom)

    y_h2_top = y_start - ROW_HEIGHT
    curr_x = x_start + COL_WIDTHS[0]
    skip_indices = [3, 5, 9] 
    
    for i in range(1, 12):
        curr_x += COL_WIDTHS[i]
        if i not in [6, 11] and i not in skip_indices: 
             c.line(curr_x, y_h2_top, curr_x, y_bottom)

    # Divisores Internos
    for i in range(3):
        y_top_cell = y_data_start - (i * ROW_HEIGHT)
        y_mid_cell = y_top_cell - (ROW_HEIGHT / 2)
        y_bot_cell = y_top_cell - ROW_HEIGHT
        
        cx = x_start + sum(COL_WIDTHS[:3]) + COL_WIDTHS[3]; c.line(cx, y_mid_cell, cx, y_bot_cell) 
        cx = x_start + sum(COL_WIDTHS[:5]) + COL_WIDTHS[5]; c.line(cx, y_mid_cell, cx, y_bot_cell)
        cx = x_start + sum(COL_WIDTHS[:9]) + COL_WIDTHS[9]; c.line(cx, y_mid_cell, cx, y_bot_cell)


def popular_grupos123(c, x_start, y_start):
    cabecalhos = ["Grupos", "Pickup", "Seq", "Curva Lenta", "Curva Rápida", "Pickup", "Seq", "Curva Lenta", "Curva Rápida", "Tempo Morto"]
    
    c.setFont("Helvetica-Bold", 8)
    y_h1 = y_start - ROW_HEIGHT/2 - 2.5
    c.drawCentredString(x_start + COL_WIDTHS[0]/2, y_h1, "Grupos") 
    c.drawCentredString(x_start + COL_WIDTHS[0] + sum(COL_WIDTHS[1:7])/2, y_h1, "FASE")
    c.drawCentredString(x_start + sum(COL_WIDTHS[:7]) + sum(COL_WIDTHS[7:12])/2, y_h1, "TERRA")
    c.drawCentredString(x_start + sum(COL_WIDTHS[:12]) + COL_WIDTHS[12]/2, y_h1, "Tempo Morto")

    c.setFont("Helvetica-Bold", 7)
    y_h2 = y_start - 1.5*ROW_HEIGHT - 2.5
    headers_map = [
        ("Pickup", 1, 1), ("Seq", 2, 1), ("Curva Lenta", 3, 2), ("Curva Rápida", 5, 2), 
        ("Pickup", 7, 1), ("Seq", 8, 1), ("Curva Lenta", 9, 2), ("Curva Rápida", 11, 1)
    ]
    for text, start_idx, span in headers_map:
        x_pos = x_start + sum(COL_WIDTHS[:start_idx])
        w_span = sum(COL_WIDTHS[start_idx : start_idx + span])
        c.drawCentredString(x_pos + w_span/2, y_h2, text)

    # --- DADOS ---
    y_data_start = y_start - 2 * ROW_HEIGHT
    curvas_fase = ["IEC MUITO INVERSO", "IEC EXT. INVERSO", "T. DEFINIDO"]
    
    for i in range(3):
        y_top = y_data_start - (i * ROW_HEIGHT)
        y_mid = y_top - ROW_HEIGHT/2
        y_bot = y_top - ROW_HEIGHT
        
        c.setFont("Helvetica-Bold", 9)
        c.drawCentredString(x_start + COL_WIDTHS[0]/2, y_mid - 3, f"Grupo {i+1}")
        
        c.setFont("Helvetica", 8)
        # FASE Pk/Seq
        c.drawCentredString(x_start + COL_WIDTHS[0] + COL_WIDTHS[1]/2, y_mid - 3, str(random.randint(100, 400)))
        c.drawCentredString(x_start + sum(COL_WIDTHS[:2]) + COL_WIDTHS[2]/2, y_mid - 3, "3L")
        
        cl_f = random.choice(curvas_fase); cr_f = random.choice(["IEC INVERSO", ""])
        
        # AJUSTE FINO DE ALINHAMENTO VERTICAL (Dial e T. Adic)
        y_label = y_mid - 2  # Labels (Dial) um pouco mais pra cima
        y_val = y_bot + ROW_HEIGHT*0.25 - 3 # Valores centralizados na caixa de baixo

        # FASE CL
        x_base = x_start + sum(COL_WIDTHS[:3])
        c.drawCentredString(x_base + (COL_WIDTHS[3]+COL_WIDTHS[4])/2, y_top - ROW_HEIGHT*0.25 - 3, cl_f)
        if cl_f != "T. DEFINIDO":
            c.setFont("Helvetica", 6); c.drawCentredString(x_base + COL_WIDTHS[3]/2, y_label, "Dial"); c.drawCentredString(x_base + COL_WIDTHS[3] + COL_WIDTHS[4]/2, y_label, "T. Adic.")
            c.setFont("Helvetica", 8); c.drawCentredString(x_base + COL_WIDTHS[3]/2, y_val, f"{random.uniform(0.1, 1.0):.2f}".replace('.',','))

        # FASE CR
        x_base = x_start + sum(COL_WIDTHS[:5])
        c.drawCentredString(x_base + (COL_WIDTHS[5]+COL_WIDTHS[6])/2, y_top - ROW_HEIGHT*0.25 - 3, cr_f)
        if cr_f:
             c.setFont("Helvetica", 6); c.drawCentredString(x_base + COL_WIDTHS[5]/2, y_label, "Dial")
             c.setFont("Helvetica", 8); c.drawCentredString(x_base + COL_WIDTHS[5]/2, y_val, f"{random.uniform(0.1, 1.0):.2f}".replace('.',','))

        # TERRA Pk/Seq
        x_base = x_start + sum(COL_WIDTHS[:7])
        c.drawCentredString(x_base + COL_WIDTHS[7]/2, y_mid - 3, str(random.randint(40, 80)))
        c.drawCentredString(x_base + COL_WIDTHS[7] + COL_WIDTHS[8]/2, y_mid - 3, "3L")

        # TERRA CL
        x_base = x_start + sum(COL_WIDTHS[:9])
        c.drawCentredString(x_base + (COL_WIDTHS[9]+COL_WIDTHS[10])/2, y_top - ROW_HEIGHT*0.25 - 3, "T. DEFINIDO")
        c.setFont("Helvetica", 6); c.drawCentredString(x_base + COL_WIDTHS[9] + COL_WIDTHS[10]/2, y_label, "Tempo")
        c.setFont("Helvetica", 8); c.drawCentredString(x_base + COL_WIDTHS[9] + COL_WIDTHS[10]/2, y_val, str(random.randint(10, 30)))

        # TERRA CR
        x_base = x_start + sum(COL_WIDTHS[:11])
        c.setFont("Helvetica", 6); c.drawCentredString(x_base + COL_WIDTHS[11]/2, y_label, "Tempo")
        val = str(random.randint(5, 15)) if random.random() > 0.5 else "-"
        c.setFont("Helvetica", 8); c.drawCentredString(x_base + COL_WIDTHS[11]/2, y_val, val)

        # TM
        x_base = x_start + sum(COL_WIDTHS[:12])
        vals = [str(random.choice([10, 20])) for _ in range(random.randint(1, 3))]
        c.drawCentredString(x_base + COL_WIDTHS[12]/2, y_mid - 3, ", ".join(list(set(vals))))

def desenhar_grupo4_linear(c, x_start, y_start):
    y_top = y_start
    y_bottom = y_start - ROW_HEIGHT
    
    # Grupo
    c.rect(x_start, y_bottom, COL_WIDTHS[0], ROW_HEIGHT)
    c.setFont("Helvetica-Bold", 9)
    c.drawCentredString(x_start + COL_WIDTHS[0]/2, y_bottom + ROW_HEIGHT/2 - 4, "Grupo 4")
    
    curr_x = x_start + COL_WIDTHS[0]
    widths = [COL_WIDTHS[1], COL_WIDTHS[2], COL_WIDTHS[3]+COL_WIDTHS[4], COL_WIDTHS[5], COL_WIDTHS[6],
              COL_WIDTHS[7], COL_WIDTHS[8], COL_WIDTHS[9]+COL_WIDTHS[10], COL_WIDTHS[11]]
    
    dados = ["630", "1L", "T. DEFINIDO", "Lenta", "20", "630", "1L", "T. DEFINIDO", "Lenta"]

    c.setFont("Helvetica", 9)
    for i, dado in enumerate(dados):
        w = widths[i]
        c.rect(curr_x, y_bottom, w, ROW_HEIGHT)
        c.drawCentredString(curr_x + w/2, y_bottom + ROW_HEIGHT/2 - 4, dado)
        curr_x += w
    
    c.rect(curr_x, y_bottom, COL_WIDTHS[12], ROW_HEIGHT)
    c.drawCentredString(curr_x + COL_WIDTHS[12]/2, y_bottom + ROW_HEIGHT/2 - 4, "20")

def gerar_pdf_modelo_avancado(nome_arquivo):
    c = canvas.Canvas(nome_arquivo, pagesize=landscape(A4))
    width, height = landscape(A4)
    
    desenhar_cabecalho(c, width, height)
    
    y_anchor = height - 60*mm
    x_anchor = (width - sum(COL_WIDTHS)) / 2

    desenhar_grade_grupos123(c, x_anchor, y_anchor)
    popular_grupos123(c, x_anchor, y_anchor)
    
    y_g4 = y_anchor - (5 * ROW_HEIGHT) - 10*mm
    desenhar_grupo4_linear(c, x_anchor, y_g4)
    
    c.setFont("Helvetica-Oblique", 8)
    c.drawString(x_anchor, 15*mm, "Documento modelo gerado para teste de robustez de ETL (Simulação de layout complexo e linear).")
    c.save()
    print(f"PDF Final gerado: {nome_arquivo}")

if __name__ == "__main__":
    output_folder = "pdfs_ficticios_final_v3"
    if not os.path.exists(output_folder): os.makedirs(output_folder)
    print("Gerando 5 PDFs modelo...")
    for i in range(1, 6):
        nome = os.path.join(output_folder, f"Relatorio_Tecnico_EnergyCorp_{i:03d}.pdf")
        gerar_pdf_modelo_avancado(nome)

Gerando 5 PDFs modelo...
PDF Final gerado: pdfs_ficticios_final_v3\Relatorio_Tecnico_EnergyCorp_001.pdf
PDF Final gerado: pdfs_ficticios_final_v3\Relatorio_Tecnico_EnergyCorp_002.pdf
PDF Final gerado: pdfs_ficticios_final_v3\Relatorio_Tecnico_EnergyCorp_003.pdf
PDF Final gerado: pdfs_ficticios_final_v3\Relatorio_Tecnico_EnergyCorp_004.pdf
PDF Final gerado: pdfs_ficticios_final_v3\Relatorio_Tecnico_EnergyCorp_005.pdf


In [1]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.units import mm
from reportlab.lib import colors
import random
import os

# ================= CONFIGURAÇÕES GERAIS =================
# Definição EXATA das 13 colunas
COL_WIDTHS = [
    22*mm, # 0: Grupo
    18*mm, # 1: Pk F
    14*mm, # 2: Sq F
    28*mm, # 3: CL F (Nome)
    14*mm, # 4: CL F (Dial)
    28*mm, # 5: CR F (Nome)
    14*mm, # 6: CR F (Dial)
    18*mm, # 7: Pk T
    14*mm, # 8: Sq T
    28*mm, # 9: CL T (Nome)
    14*mm, # 10: CL T (Tempo)
    24*mm, # 11: CR T (Tempo - Coluna Única)
    18*mm  # 12: TM
]
ROW_HEIGHT = 16*mm 

def desenhar_cabecalho(c, width, height):
    """Desenha o cabeçalho fake da empresa"""
    c.setFont("Helvetica-Bold", 14)
    c.drawString(15*mm, height - 20*mm, "ENERGYCORP S.A. - DISTRIBUIÇÃO")
    
    c.setFont("Helvetica-Bold", 10)
    c.setFillColor(colors.lightgrey)
    c.rect(15*mm, height - 35*mm, width - 30*mm, 8*mm, fill=1, stroke=0)
    c.setFillColor(colors.black)
    c.drawCentredString(width/2, height - 31*mm, "SUMÁRIO DE AJUSTES DE RELIGADOR (MODELO FICTÍCIO)")
    
    c.setFont("Helvetica", 9)
    y_data = height - 45*mm
    c.drawString(15*mm, y_data, f"Alimentador: ALIM-{random.randint(100,999)}")
    c.drawString(120*mm, y_data, f"Coord.: {random.randint(10,99)} {random.randint(100000,999999)}:{random.randint(1000000,9999999)}")
    c.drawString(width - 60*mm, y_data, f"Data: {random.randint(1,28)}/0{random.randint(1,9)}/2024")
    
    c.setStrokeColor(colors.black)
    c.setLineWidth(1)
    c.line(15*mm, y_data - 5*mm, width - 15*mm, y_data - 5*mm)

def desenhar_grade_grupos123(c, x_start, y_start):
    """Desenha a estrutura da grade"""
    total_w = sum(COL_WIDTHS)
    y_bottom = y_start - (5 * ROW_HEIGHT)
    
    c.setLineWidth(0.5)
    c.setStrokeColor(colors.black)
    c.rect(x_start, y_bottom, total_w, 5 * ROW_HEIGHT)
    
    # --- Linhas Horizontais Principais ---
    for i in range(1, 5):
        y = y_start - (i * ROW_HEIGHT)
        c.line(x_start, y, x_start + total_w, y)

    # --- Linhas Horizontais Intermediárias (Sub-divisões) ---
    y_data_start = y_start - (2 * ROW_HEIGHT)
    for i in range(3):
        y_mid = y_data_start - (i * ROW_HEIGHT) - (ROW_HEIGHT / 2)
        
        # FASE (Cols 3 a 6)
        x_fase = x_start + sum(COL_WIDTHS[:3])
        c.line(x_fase, y_mid, x_fase + sum(COL_WIDTHS[3:7]), y_mid)
        # TERRA CL (Cols 9 e 10)
        x_terra_cl = x_start + sum(COL_WIDTHS[:9])
        c.line(x_terra_cl, y_mid, x_terra_cl + COL_WIDTHS[9] + COL_WIDTHS[10], y_mid)

    # --- Linhas Verticais ---
    c.line(x_start + COL_WIDTHS[0], y_start, x_start + COL_WIDTHS[0], y_bottom)
    c.line(x_start + sum(COL_WIDTHS[:7]), y_start, x_start + sum(COL_WIDTHS[:7]), y_bottom)
    c.line(x_start + sum(COL_WIDTHS[:12]), y_start, x_start + sum(COL_WIDTHS[:12]), y_bottom)

    y_h2_top = y_start - ROW_HEIGHT
    curr_x = x_start + COL_WIDTHS[0]
    skip_indices = [3, 5, 9] 
    
    for i in range(1, 12):
        curr_x += COL_WIDTHS[i]
        if i not in [6, 11] and i not in skip_indices: 
             c.line(curr_x, y_h2_top, curr_x, y_bottom)

    # Divisores Internos
    for i in range(3):
        y_top_cell = y_data_start - (i * ROW_HEIGHT)
        y_mid_cell = y_top_cell - (ROW_HEIGHT / 2)
        y_bot_cell = y_top_cell - ROW_HEIGHT
        
        cx = x_start + sum(COL_WIDTHS[:3]) + COL_WIDTHS[3]; c.line(cx, y_mid_cell, cx, y_bot_cell) 
        cx = x_start + sum(COL_WIDTHS[:5]) + COL_WIDTHS[5]; c.line(cx, y_mid_cell, cx, y_bot_cell)
        cx = x_start + sum(COL_WIDTHS[:9]) + COL_WIDTHS[9]; c.line(cx, y_mid_cell, cx, y_bot_cell)


def popular_grupos123(c, x_start, y_start):
    cabecalhos = ["Grupos", "Pickup", "Seq", "Curva Lenta", "Curva Rápida", "Pickup", "Seq", "Curva Lenta", "Curva Rápida", "Tempo Morto"]
    
    c.setFont("Helvetica-Bold", 8)
    y_h1 = y_start - ROW_HEIGHT/2 - 2.5
    c.drawCentredString(x_start + COL_WIDTHS[0]/2, y_h1, "Grupos") 
    c.drawCentredString(x_start + COL_WIDTHS[0] + sum(COL_WIDTHS[1:7])/2, y_h1, "FASE")
    c.drawCentredString(x_start + sum(COL_WIDTHS[:7]) + sum(COL_WIDTHS[7:12])/2, y_h1, "TERRA")
    c.drawCentredString(x_start + sum(COL_WIDTHS[:12]) + COL_WIDTHS[12]/2, y_h1, "Tempo Morto")

    c.setFont("Helvetica-Bold", 7)
    y_h2 = y_start - 1.5*ROW_HEIGHT - 2.5
    headers_map = [
        ("Pickup", 1, 1), ("Seq", 2, 1), ("Curva Lenta", 3, 2), ("Curva Rápida", 5, 2), 
        ("Pickup", 7, 1), ("Seq", 8, 1), ("Curva Lenta", 9, 2), ("Curva Rápida", 11, 1)
    ]
    for text, start_idx, span in headers_map:
        x_pos = x_start + sum(COL_WIDTHS[:start_idx])
        w_span = sum(COL_WIDTHS[start_idx : start_idx + span])
        c.drawCentredString(x_pos + w_span/2, y_h2, text)

    # --- DADOS ---
    y_data_start = y_start - 2 * ROW_HEIGHT
    curvas_fase = ["IEC MUITO INVERSO", "IEC EXT. INVERSO", "T. DEFINIDO"]
    
    for i in range(3):
        y_top = y_data_start - (i * ROW_HEIGHT)
        y_mid = y_top - ROW_HEIGHT/2
        y_bot = y_top - ROW_HEIGHT
        
        # Coordenadas Y centrais para as sub-células (Matemática exata)
        y_upper_cell = y_top - (ROW_HEIGHT/4) - 2 # Centro da metade de cima
        y_lower_cell = y_mid - (ROW_HEIGHT/4) - 2 # Centro da metade de baixo
        
        c.setFont("Helvetica-Bold", 9)
        c.drawCentredString(x_start + COL_WIDTHS[0]/2, y_mid - 3, f"Grupo {i+1}")
        
        c.setFont("Helvetica", 8)
        # FASE Pk/Seq
        c.drawCentredString(x_start + COL_WIDTHS[0] + COL_WIDTHS[1]/2, y_mid - 3, str(random.randint(100, 400)))
        c.drawCentredString(x_start + sum(COL_WIDTHS[:2]) + COL_WIDTHS[2]/2, y_mid - 3, "3L")
        
        cl_f = random.choice(curvas_fase); cr_f = random.choice(["IEC INVERSO", ""])
        
        # FASE CL
        x_base = x_start + sum(COL_WIDTHS[:3])
        # Nome da Curva (Centralizado na parte de cima)
        c.drawCentredString(x_base + (COL_WIDTHS[3]+COL_WIDTHS[4])/2, y_upper_cell, cl_f)
        
        if cl_f != "T. DEFINIDO":
            # Labels (Dial / T.Adic)
            c.setFont("Helvetica", 6)
            c.drawCentredString(x_base + COL_WIDTHS[3]/2, y_mid - 3, "Dial") # Um pouco acima da linha divisoria
            c.drawCentredString(x_base + COL_WIDTHS[3] + COL_WIDTHS[4]/2, y_mid - 3, "T. Adic.")
            # Valores (Centralizados na parte de baixo)
            c.setFont("Helvetica", 8)
            c.drawCentredString(x_base + COL_WIDTHS[3]/2, y_lower_cell, f"{random.uniform(0.1, 1.0):.2f}".replace('.',','))

        # FASE CR
        x_base = x_start + sum(COL_WIDTHS[:5])
        c.drawCentredString(x_base + (COL_WIDTHS[5]+COL_WIDTHS[6])/2, y_upper_cell, cr_f)
        if cr_f:
             c.setFont("Helvetica", 6)
             c.drawCentredString(x_base + COL_WIDTHS[5]/2, y_mid - 3, "Dial")
             c.setFont("Helvetica", 8)
             c.drawCentredString(x_base + COL_WIDTHS[5]/2, y_lower_cell, f"{random.uniform(0.1, 1.0):.2f}".replace('.',','))

        # TERRA Pk/Seq
        x_base = x_start + sum(COL_WIDTHS[:7])
        c.drawCentredString(x_base + COL_WIDTHS[7]/2, y_mid - 3, str(random.randint(40, 80)))
        c.drawCentredString(x_base + COL_WIDTHS[7] + COL_WIDTHS[8]/2, y_mid - 3, "3L")

        # TERRA CL
        x_base = x_start + sum(COL_WIDTHS[:9])
        c.drawCentredString(x_base + (COL_WIDTHS[9]+COL_WIDTHS[10])/2, y_upper_cell, "T. DEFINIDO")
        
        c.setFont("Helvetica", 6)
        c.drawCentredString(x_base + COL_WIDTHS[9] + COL_WIDTHS[10]/2, y_mid - 3, "Tempo") # Label centralizado
        
        c.setFont("Helvetica", 8)
        c.drawCentredString(x_base + COL_WIDTHS[9] + COL_WIDTHS[10]/2, y_lower_cell, str(random.randint(10, 30)))

        # TERRA CR (Coluna Única)
        x_base = x_start + sum(COL_WIDTHS[:11])
        c.setFont("Helvetica", 6)
        c.drawCentredString(x_base + COL_WIDTHS[11]/2, y_mid - 3, "Tempo")
        
        c.setFont("Helvetica", 8)
        val = str(random.randint(5, 15)) if random.random() > 0.5 else "-"
        c.drawCentredString(x_base + COL_WIDTHS[11]/2, y_lower_cell, val) # Alinhado com os outros valores

        # TM
        x_base = x_start + sum(COL_WIDTHS[:12])
        vals = [str(random.choice([10, 20])) for _ in range(random.randint(1, 3))]
        c.drawCentredString(x_base + COL_WIDTHS[12]/2, y_mid - 3, ", ".join(list(set(vals))))

def desenhar_grupo4_linear(c, x_start, y_start):
    y_top = y_start
    y_bottom = y_start - ROW_HEIGHT
    
    # Grupo
    c.rect(x_start, y_bottom, COL_WIDTHS[0], ROW_HEIGHT)
    c.setFont("Helvetica-Bold", 9)
    c.drawCentredString(x_start + COL_WIDTHS[0]/2, y_bottom + ROW_HEIGHT/2 - 4, "Grupo 4")
    
    curr_x = x_start + COL_WIDTHS[0]
    widths = [COL_WIDTHS[1], COL_WIDTHS[2], COL_WIDTHS[3]+COL_WIDTHS[4], COL_WIDTHS[5], COL_WIDTHS[6],
              COL_WIDTHS[7], COL_WIDTHS[8], COL_WIDTHS[9]+COL_WIDTHS[10], COL_WIDTHS[11]]
    
    dados = ["630", "1L", "T. DEFINIDO", "Lenta", "20", "630", "1L", "T. DEFINIDO", "Lenta"]

    c.setFont("Helvetica", 9)
    for i, dado in enumerate(dados):
        w = widths[i]
        c.rect(curr_x, y_bottom, w, ROW_HEIGHT)
        c.drawCentredString(curr_x + w/2, y_bottom + ROW_HEIGHT/2 - 4, dado)
        curr_x += w
    
    c.rect(curr_x, y_bottom, COL_WIDTHS[12], ROW_HEIGHT)
    c.drawCentredString(curr_x + COL_WIDTHS[12]/2, y_bottom + ROW_HEIGHT/2 - 4, "20")

def gerar_pdf_modelo_avancado(nome_arquivo):
    c = canvas.Canvas(nome_arquivo, pagesize=landscape(A4))
    width, height = landscape(A4)
    desenhar_cabecalho(c, width, height)
    
    y_anchor = height - 60*mm
    x_anchor = (width - sum(COL_WIDTHS)) / 2

    desenhar_grade_grupos123(c, x_anchor, y_anchor)
    popular_grupos123(c, x_anchor, y_anchor)
    
    y_g4 = y_anchor - (5 * ROW_HEIGHT) - 10*mm
    desenhar_grupo4_linear(c, x_anchor, y_g4)
    
    c.setFont("Helvetica-Oblique", 8)
    c.drawString(x_anchor, 15*mm, "Documento modelo gerado para teste de robustez de ETL (Simulação de layout complexo e linear).")
    c.save()
    print(f"PDF Final gerado: {nome_arquivo}")

if __name__ == "__main__":
    output_folder = "pdfs_ficticios_final_v4"
    if not os.path.exists(output_folder): os.makedirs(output_folder)
    print("Gerando 50 PDFs modelo para carga de teste...")
    for i in range(1, 51):
        nome = os.path.join(output_folder, f"Relatorio_Tecnico_EnergyCorp_{i:03d}.pdf")
        gerar_pdf_modelo_avancado(nome)
    print("Concluído! Verifique a pasta 'pdfs_ficticios_final_v4'.")

Gerando 50 PDFs modelo para carga de teste...
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_001.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_002.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_003.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_004.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_005.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_006.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_007.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_008.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_009.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_010.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_011.pdf
PDF Final gerado: pdfs_ficticios_final_v4\Relatorio_Tecnico_EnergyCorp_012.pdf
PDF Fi